Artifical Start

In [10]:
using LinearAlgebra
using SparseArrays
using LightGraphs
using GraphPlot
using BenchmarkTools
include("minCost.jl")

AddToTable!

In [11]:
nodes = 13
# t nodes 1-5
# buy nodes 6-9
# laundry nodes 10-12
# dump node 13

E = [(1,2),(2,3),(3,4),(4,5),(5,13),(6,2),(6,13),(7,3),(7,13),(8,4),(8,13),(9,5),(9,14),(10,3),(10,4),(10,5),(10,13),(11,4),(11,5),(11,13),(12,5),(12,13)]

# cost of services 
c0 = 3
c1 = 5
c2 = 2
c3 = 1

cost_dict = Dict((1,2)=>0,(2,3)=>0,(3,4)=>0,(4,5)=>0,(5,13)=>0,
    (6,2)=>c0,(6,13)=>0,(7,3)=>c0,(7,13)=>0,(8,4)=>c0,(8,13)=>0,(9,5)=>c0,(9,14)=>0,
    (10,3)=>c1,(10,4)=>c2,(10,5)=>c3,(10,13)=>0,
    (11,4)=>c1,(11,5)=>c2,(11,13)=>0,
    (12,5)=>c1,(12,13)=>0)
nodelabel = ["t_0", "t_1","t_2","t_3","t_4","b_1","b_2","b_3","b_4","L_1","L_2","L_3","D"]
c = ones(Int, 14,1)
c[14] = 0
# napkins
r0 = 25
r1 = 50
r2 = 75
r3 = 85
r4 = 100

b = Int64[r0; -r1; -r2; -r3; -r4; r1; r2; r3; r1; r2; r3; r4; -260; 0]
;

In [ ]:
g = SimpleDiGraph(nodes)
for e in E
    add_edge!(g,e)
end

gplot(g, layout=shell_layout, nodelabel=nodelabel)

In [3]:
n = nodes + 1 
x = abs.(b)
B = ArtificalBase_table(b)
B_edgeList = ArtificalBase_list(b)
B_matrix = BaseMatrix(B_edgeList,n)
NB_edgeList = copy(E);

LoadError: UndefVarError: ArtificalBase_table not defined

In [4]:
w = zeros(n,1)
valid = true
while (sum(c) != 0)
    w = c'/B_matrix    
    max ,enteringEdge = ReducedCost(w,NB_edgeList,1, 0)
    
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,n,B)
    Flow_list, valid = Get_flow(path)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,n,B)
        RemoveFromTable!(edge_j,edge_i,n,B)
        AddToTable!(edge_u, edge_v,1,B,n)
        AddToTable!(edge_v, edge_u,-1,B,n)
        Update_BaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        c[min_index] = 0
    end
    for (i,e) in enumerate(NB_edgeList) 
        if e == enteringEdge
            NB_edgeList[i] = (0,0)
            break
        end
    end
end

LoadError: UndefVarError: B_matrix not defined

In [5]:
basicEdge =  Base_list(B_edgeList, nodes)
cost = Set_cost(basicEdge, cost_dict)
NB_edgeList = NonBasic_list(basicEdge, E, nodes)
B_matrix = BaseMatrix(basicEdge,nodes)
B = Base_table(basicEdge, nodes)

b = b[1:end-1] # Flow
x = x[1:end-1];

LoadError: UndefVarError: Base_list not defined

In [6]:
w = cost'/B_matrix
while (w*cost != 0)
    max ,enteringEdge = ReducedCost(w,NB_edgeList, 2, cost_dict)
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,nodes,B)
    Flow_list, valid = Get_flow(path)
    if valid
        indexes = Get_Indexes(path,basicEdge) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,nodes,B)
        RemoveFromTable!(edge_j,edge_i,nodes,B)
        AddToTable!(edge_u, edge_v,1,B, nodes)
        AddToTable!(edge_v, edge_u,-1,B, nodes)
        Update_BaseMatrix!(B_matrix,basicEdge,exitingEdge,enteringEdge)
        cost[min_index] = cost_dict[enteringEdge]
    end
    w = cost'/B_matrix
end

LoadError: UndefVarError: cost not defined

In [7]:
# Check optimality through the KKT
λ_b = cost' - w*B_matrix
@show norm(B_matrix*x - b)
@show minimum(x) >= 0
@show norm(λ_b);

LoadError: UndefVarError: cost not defined

In [8]:
g_optimal = SimpleDiGraph(nodes)
println("Cost: ", dot(x,cost))
for i in 1:nodes
    if x[i] > 0
        println("$(basicEdge[i]) : $(x[i])  $(cost[i])")
        add_edge!(g_optimal,basicEdge[i])
    end
end

LoadError: UndefVarError: SimpleDiGraph not defined

In [9]:
gplot(g_optimal, layout=shell_layout, nodelabel=nodelabel)

LoadError: UndefVarError: shell_layout not defined